# Time series prediction, end-to-end
This notebook illustrates several models to find the next value of a time-series:

1. __Linear__
2. __DNN__
3. __CNN__
4. __RNN__

In [2]:
SEQ_LEN = 50

## Simulate some time-series data
Essentially a set of sinusoids with random amplitudes and frequencies.

In [1]:
import os
import tensorflow as tf
print(tf.__version__)

H:\anaconda_installed\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
H:\anaconda_installed\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
H:\anaconda_installed\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
H:\anaconda_installed\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1)

1.14.0


In [3]:
import numpy as np
import seaborn as sns

def create_time_series():
  freq = (np.random.random()*0.5) + 0.1  # 0.1 to 0.6
  ampl = np.random.random() + 0.5  # 0.5 to 1.5
  noise = [np.random.random()*0.3 for i in range(SEQ_LEN)] # -0.3 to +0.3 uniformly distributed
  x = np.sin(np.arange(0,SEQ_LEN) * freq) * ampl + noise
  return x

flatui = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
for i in range(0, 5):
  sns.tsplot( create_time_series(), color=flatui[i%len(flatui)] );  # 5 series

H:\anaconda_installed\lib\site-packages\seaborn\timeseries.py:183: UserWarning: The tsplot function is deprecated and will be removed or replaced (in a substantially altered version) in a future release.
  warnings.warn(msg, UserWarning)
H:\anaconda_installed\lib\site-packages\scipy\stats\stats.py:1633: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [4]:
def to_csv(filename, N):
  with open(filename, 'w') as ofp:
    for lineno in range(0, N):
      seq = create_time_series()
      line = ",".join(map(str, seq))
      ofp.write(line + '\n')

import os
try:
  os.makedirs('data/sines/')
except OSError:
  pass

np.random.seed(1) # makes data generation reproducible

to_csv('data/sines/train-1.csv', 1000)  # 1000 sequences
to_csv('data/sines/valid-1.csv', 250)

'head' is not recognized as an internal or external command,
operable program or batch file.


## Train model locally
Make sure the code works as intended.

In [9]:
%%bash
DATADIR=$(pwd)/data/sines
OUTDIR=$(pwd)/trained/sines
gcloud ml-engine local train \
   --module-name=sinemodel.task \
   --package-path=${PWD}/sinemodel \
   -- \
   --train_data_path="${DATADIR}/train-1.csv" \
   --eval_data_path="${DATADIR}/valid-1.csv"  \
   --output_dir=${OUTDIR} \
   --model=linear --train_steps=10 --sequence_length=$SEQ_LEN

Couldn't find program: 'bash'


In [29]:
%run -i task.py --train_data_path="./data/sines/train-1.csv" --eval_data_path="./data/sines/valid-1.csv" --output_dir="OUTDIR" --model="linear" --train_steps=10 --sequence_length=50

I0728 19:32:27.034844 11388 keras.py:424] Using the Keras model provided.
I0728 19:32:27.559135 11388 estimator.py:209] Using config: {'_model_dir': 'OUTDIR\\', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 60, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002278A587CC0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
I0728 19:32:27.563156 11388 estimat

NotFoundError: Failed to create a NewWriteableFile: OUTDIR\export\exporter\temp-b'1564324350'\variables\variables_temp_8c3e2536bcba41af950ec55eca7f9a04/part-00000-of-00001.data-00000-of-00001.tempstate4639463215713247737 : The system cannot find the path specified.
; No such process
	 [[node save/SaveV2 (defined at F:\maaz\Coursera\Advance Machine Learning with TensorFlow on GCP Specialization\Python Pacakge for Training\Keras\model.py:198) ]]

Original stack trace for 'save/SaveV2':
  File "H:\anaconda_installed\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "H:\anaconda_installed\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "H:\anaconda_installed\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "H:\anaconda_installed\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "H:\anaconda_installed\lib\site-packages\ipykernel\ker